# CSV_TO_SQL CONVERTION USING POSTGRES

I am currently working on a data analysis project which involves large csv file,100MB to 6 GB.If we try to load this amount of data in a pandas dataframe, it fails due to memory limitation of a home computer or a laptop.<br>
Solution to this is to convert the csv files to sql and then read the sql database in chunks in pandas to conduct our data analysis.<br>
This tutorial is useful for anyone who is working on Windows/python and Anaconda framework to conduct the research.
    

## My Local Machine configration:
Microsoft Windows [Version 10.0.14393]<br>
Python 2.7.12 |Anaconda 4.2.0 (64-bit)<br>
INSTALLED RAM : 4 GB<br>
HARD DISK: 920 GB<br>

### Using Postgres Database And pscopg2 library to convert csv to postgres.


## Useful command in postgres sql
|command|Description what it does|
|---|----------------------------|
|\l  |lists all the database|   
|\c dbname|connect to the database named dbname|
|\dt|list the table in the database you have connected|
|select * from tablename;|display all the record from table named tablename|
|delete from tablename;|delete all record in the table named tablename.|
|show data_directory;|show the curent working directory in postgres|


In [1]:
# to check the no of rows for big table,rel name is the name of the table.
#SELECT reltuples::bigint AS estimate FROM pg_class where relname='transaction';


In [2]:
# create a database named testing in postgres
#create database test;
import psycopg2
conn = psycopg2.connect(host="localhost",dbname="postgres",user="postgres",password="ram501shi")

# this statement will at commit all statements
conn.autocommit=True
cur=conn.cursor()
dname='test'
cur.execute('CREATE DATABASE '+dname)
print("Database '"+dname+"' was created sucessfully.")


Database 'test' was created sucessfully.


In [ ]:
# %load ./data/db1/data.csv
store_id	state_code	post_code	group_store\n
1	VIC	3558	0
2	VIC	3124	0
3	WA	6107	1
4	VIC	3214	0
5	VIC	3172	0
6	NSW	2089	0
7	NSW	2460	0
8	VIC	3107	0
9	VIC	3130	0

In [8]:
conn = psycopg2.connect(host="localhost",database="test",user="postgres",password="ram501shi")
# this statement will at commit all statements
conn.autocommit=True
cur=conn.cursor()
cur.execute('create table store(store_id integer primary key not null,"state_code" text null,post_code integer not null,group_store bool not null);')
print("Table store created")
cur.close()
conn.close()



Table store created


In [9]:
conn = psycopg2.connect(host="localhost",database="test",user="postgres",password="ram501shi")
# this statement will at commit all statements
conn.autocommit=True
cur=conn.cursor()
cur.execute("select * from store;")
data=cur.fetchall()
print data

[]


<img src="./figures/sc2.png">


In [10]:
import psycopg2
import glob 
conn = psycopg2.connect(host="localhost",database="test",user="postgres",password="ram501shi")
# this statement will at commit all statements
conn.autocommit=True
# we use r prefix to indicate we want the data raw.Abosolute path or relative path can be used just have to get the path right.
# we use relative path ie from   C:/Program Files/PostgreSQL/9.6/data where the database is located to where the csv file is 
# located on the local machine. ../../../../will bring us to C:// then we navigate to the data location 
# ie /Users/SwapnaDev/Desktop/DataScience-Project/CSV_TO_SQL_CONVERTION/data/test.csv'
file=glob.glob(r'../../../../../Users/SwapnaDev/Desktop/DataScience-Project/CSV_TO_SQL_CONVERTION/data/db1/data.csv')
print"This is the location of the file :",file[0]
# constructing the sql statement to pass
# "copy store(store_id,state_code,post_code,group_store) from " copy command followed by the database name "store" and its filed
# in the brackers from 
# " +"'"+file+"'"+ : double quotes used as escape squence for single quote which is reqiured for the file path
#DELIMITER E'\\t' csv header;" :: delimiter of the file ie tab E\\t E is required and \ escape seq for \t followed by csv header. 
sql="copy store(store_id,state_code,post_code,group_store) from " +"'"+file[0]+"'"+" DELIMITER E'\\t' csv header;"
print sql
cur=conn.cursor()
cur.execute(sql)


This is the location of the file : ../../../../../Users/SwapnaDev/Desktop/DataScience-Project/CSV_TO_SQL_CONVERTION/data/db1/data.csv
copy store(store_id,state_code,post_code,group_store) from '../../../../../Users/SwapnaDev/Desktop/DataScience-Project/CSV_TO_SQL_CONVERTION/data/db1/data.csv' DELIMITER E'\t' csv header;


In [11]:
import psycopg2
conn = psycopg2.connect(host="localhost",database="test",user="postgres",password="ram501shi")
cu=conn.cursor()
cu.execute("select * from store;")
a=cu.fetchall()
print "Checking if the data was added from the test.csv file to the database."
for item in a:
    print item


Checking if the data was added from the test.csv file to the database.
(1, 'VIC', 3558, False)
(2, 'VIC', 3124, False)
(3, 'WA', 6107, True)
(4, 'VIC', 3214, False)
(5, 'VIC', 3172, False)
(6, 'NSW', 2089, False)
(7, 'NSW', 2460, False)
(8, 'VIC', 3107, False)
(9, 'VIC', 3130, False)


Also screenshot from the psql prompt.
<img src='./figures/sc3.png'>

## Now we want convert multiple file in csv format in to SQL table.

In [12]:
# import glob to use it to get all the file name in a directory.
# there are 2 file in this directory we want to convert to sql database.
#import glob module
import glob
# A list, named "file" having the paths of file we want to convert csv to sql. 
file=glob.glob(r'../../../../../Users/SwapnaDev/Desktop/DataScience-Project/CSV_TO_SQL_CONVERTION/data/db2/*.*')


In [13]:
# print the list,named "file"
print file





['../../../../../Users/SwapnaDev/Desktop/DataScience-Project/CSV_TO_SQL_CONVERTION/data/db2\\test.csv', '../../../../../Users/SwapnaDev/Desktop/DataScience-Project/CSV_TO_SQL_CONVERTION/data/db2\\test1.csv']


In [14]:
# we notice that 2 backslash "\\" in the file path which we need to convert front slash "/" 
# define a empty list new, for storage
new=[]
# looping through list name, "file" until the last element of the list.
for all in range(len(file)):
    # replacing first item in the "list"  with the correct slash for windows file path and storing in a variable called var 
    var=file[all].replace('\\','/')
    #appending the var to a list called new.
    new.append(var)

    

In [15]:
# list with the correct path for windows
print new

['../../../../../Users/SwapnaDev/Desktop/DataScience-Project/CSV_TO_SQL_CONVERTION/data/db2/test.csv', '../../../../../Users/SwapnaDev/Desktop/DataScience-Project/CSV_TO_SQL_CONVERTION/data/db2/test1.csv']


In [16]:
# declaration a conn variable to connect to the postgres sql database
conn = psycopg2.connect(host="localhost",database="test",user="postgres",password="ram501shi")
cur=conn.cursor()



In [ ]:
# %load ./data/db2/test.csv
store_id	state_code	post_code	group_store
1	VIC	3558	0
2	VIC	3124	0
3	WA	6107	1
4	VIC	3214	0
5	VIC	3172	0
6	NSW	2089	0
7	NSW	2460	0
8	VIC	3107	0
9	VIC	3130	0

In [ ]:
# %load ./data/db2/test1.csv
store_id	state_code	post_code	group_store
10	VIC	3434	0
11	VIC	3134	0
12	WA	6007	1
13	VIC	3614	0
14	VIC	3972	0
15	NSW	2039	0
16	NSW	2460	0
17	VIC	3177	0
18	VIC	3180	0

In [21]:
#using cur.execute
#Create a database name store1 with columns name as store id,state_code,post_code,group_store and defining the variable type
conn=psycopg2.connect(host="localhost",database="test",user="postgres",password="ram501shi")
conn.autocommit=True
cur=conn.cursor()
cur.execute("create table store1(store_id integer primary key,state_code text,post_code integer,group_store bool);")
print("Table structure for "+"store1"+" created")

Table structure for store1 created


### Let us confirm that that the table "store1" exsists in psql.

In [23]:
cur.execute("select * from store1;")
data=cur.fetchall()
print("The strucure for store1 is created but it has no data in it.Hence an empty list is displayed below.")
print data


The strucure for store1 is created but it has no data in it.Hence an empty list is displayed below.
[]


We want use copy_expert method of "cursor" to convert the 2 csv file in a directory into the sql tables.
We have created the empty table named "store" with the same structure as the csv fle.

In [30]:
# we iterate through the list called "new" while length of list is less than length of the list new
for text in range(len(new)):
    connection=psycopg2.connect(host="localhost",database="test",user="postgres",password="ram501shi")
    #make auto commit to the db.
    connection.autocommit=True
    cur=connection.cursor()
    # open the file and pass the first file   
    f=open(new[text],'r')
    # construct the sql statement
    sql="copy store1(store_id,state_code,post_code,group_store) from "+"'"+new[text]+"'" +" delimiter E'\\t' csv header;"
    #print "connection file is :",f
    #print "printing sql for each file:"
    #print sql
    #pass the sql and f to the copy_expert method
    cur.copy_expert(sql,f)

    
    

   


    
        


    


<img src='./figures/sc5.png'>